<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Arthropod_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd

In [ ]:
os.environ['KAGGLE_CONFIG_DIR']='/content/drive/MyDrive/'
!kaggle datasets download -d mistag/arthropod-taxonomy-orders-object-detection-dataset
!unzip *.zip && rm *.zip